In [1]:
import pandas as pd
from datetime import datetime

class VendorAnalyticsEngine:
    def __init__(self, posts_df, ner_entities_df):
        """
        posts_df: DataFrame with columns ['vendor_id', 'post_id', 'timestamp', 'views', 'text']
        ner_entities_df: DataFrame with columns ['post_id', 'product', 'price']
        """
        self.posts_df = posts_df
        self.ner_entities_df = ner_entities_df

    def analyze_vendor(self, vendor_id):
        vendor_posts = self.posts_df[self.posts_df['vendor_id'] == vendor_id].copy()
        if vendor_posts.empty:
            return None

        # Posting Frequency (posts per week)
        vendor_posts['timestamp'] = pd.to_datetime(vendor_posts['timestamp'])
        min_date = vendor_posts['timestamp'].min()
        max_date = vendor_posts['timestamp'].max()
        weeks = max((max_date - min_date).days / 7, 1)
        posting_frequency = len(vendor_posts) / weeks

        # Average Views per Post
        avg_views = vendor_posts['views'].mean()

        # Top Performing Post
        top_post = vendor_posts.loc[vendor_posts['views'].idxmax()]
        top_post_id = top_post['post_id']
        top_post_views = top_post['views']
        top_post_text = top_post['text']

        # Get product and price for top post from NER
        top_post_ner = self.ner_entities_df[self.ner_entities_df['post_id'] == top_post_id]
        top_product = top_post_ner['product'].iloc[0] if not top_post_ner.empty else None
        top_price = top_post_ner['price'].iloc[0] if not top_post_ner.empty else None

        # Average Price Point
        vendor_post_ids = vendor_posts['post_id'].unique()
        vendor_ner = self.ner_entities_df[self.ner_entities_df['post_id'].isin(vendor_post_ids)]
        avg_price = vendor_ner['price'].astype(float).mean() if not vendor_ner.empty else None

        # Lending Score (customizable weights)
        lending_score = (avg_views * 0.5) + (posting_frequency * 0.5)

        return {
            'vendor_id': vendor_id,
            'posting_frequency_per_week': posting_frequency,
            'avg_views_per_post': avg_views,
            'top_post_id': top_post_id,
            'top_post_views': top_post_views,
            'top_product': top_product,
            'top_price': top_price,
            'avg_price_point': avg_price,
            'lending_score': lending_score
        }

    def analyze_all_vendors(self):
        vendor_ids = self.posts_df['vendor_id'].unique()
        results = []
        for vid in vendor_ids:
            res = self.analyze_vendor(vid)
            if res:
                results.append(res)
        return pd.DataFrame(results)